<a href="https://colab.research.google.com/github/vasudevgupta7/bigbird/blob/main/notebooks/evaluate_nq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue May  4 07:08:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!git clone https://github.com/vasudevgupta7/bigbird
!cd bigbird/natural-questions && pip3 install -r requirements.txt

In [ ]:
cd bigbird/natural-questions

/content/bigbird/natural-questions


In [ ]:
!wget https://huggingface.co/datasets/vasudevgupta/bigbird-tokenized-natural-questions/resolve/main/nq-val.zip
!unzip nq-val.zip

--2021-05-04 07:09:04--  https://huggingface.co/datasets/vasudevgupta/bigbird-tokenized-natural-questions/resolve/main/nq-val.zip
Resolving huggingface.co (huggingface.co)... 34.201.172.85
Connecting to huggingface.co (huggingface.co)|34.201.172.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/vasudevgupta/bigbird-tokenized-natural-questions/7209e971aef2261347768a33938ea3b1d7d741bd0bb8aa3dc803679f60ec076a [following]
--2021-05-04 07:09:04--  https://cdn-lfs.huggingface.co/datasets/vasudevgupta/bigbird-tokenized-natural-questions/7209e971aef2261347768a33938ea3b1d7d741bd0bb8aa3dc803679f60ec076a
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 52.85.144.72, 52.85.144.70, 52.85.144.56, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|52.85.144.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50599510 (48M) [application/zip]
Saving to: ‘nq-val.zip’

nq-val.zip   

In [ ]:
ls

evaluate_nq.py       nq-val.zip  prepare_nq.py     train_nq.py
nq-validation.jsonl  params.py   requirements.txt


In [ ]:
import torch
import numpy as np
from transformers import BigBirdTokenizer
from datasets import load_dataset

from params import CATEGORY_MAPPING
from train_nq import BigBirdForNaturalQuestions

CATEGORY_MAPPING = {v: k for k, v in CATEGORY_MAPPING.items()}

In [11]:
def evaluate(example, print_output=False):
    input_ids = example["input_ids"]
    input_ids = torch.tensor(input_ids, device=device, dtype=torch.long)
    input_ids.unsqueeze_(0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        _, start = outputs["start_logits"].max(dim=-1)
        _, end = outputs["end_logits"].max(dim=-1)
        start, end = start.item(), end.item()
        _, category = outputs["cls_out"].max(dim=-1)

    predicted_category = CATEGORY_MAPPING[category.item()]
    original_category = CATEGORY_MAPPING[example['category']]

    input_ids = input_ids[0].cpu().tolist()
    answer = tokenizer.decode(input_ids[start: end+1])
    orig_answer = tokenizer.decode(input_ids[example["start_token"]: example["end_token"]+1])

    if print_output:
        print(tokenizer.decode(input_ids[1:input_ids.index(tokenizer.sep_token_id)]))
        print("original_category:", original_category)
        print("predicted_category:", predicted_category)
        if original_category in ['long', 'short']:
            print("original_answer:", orig_answer)
            print("predicted_answer:", answer, end="\n\n")

    example["orig_answer"] = orig_answer
    example["answer"] = answer
    if predicted_category in ['yes', 'no', 'null']:
        example['match'] = 1 if predicted_category == original_category else 0
    else:
        example['match'] = 1 if answer == orig_answer else 0 # exact match
    return example

In [12]:
revision = "4daa96c5befa9b728c47a77280ef9484377d7791"
MODEL_ID = "vasudevgupta/bigbird-roberta-natural-questions"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = BigBirdForNaturalQuestions.from_pretrained(MODEL_ID, revision=revision).to(device)
tokenizer = BigBirdTokenizer.from_pretrained(MODEL_ID, revision=revision)

val_dataset = load_dataset("json", data_files="nq-validation.jsonl")['train']
# val_dataset = val_dataset.filter(lambda x: x['category'] != 0)
print(val_dataset)

Using custom data configuration default-64367e4e8e6b551b
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-64367e4e8e6b551b/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


Dataset({
    features: ['input_ids', 'start_token', 'end_token', 'category'],
    num_rows: 10662
})


In [ ]:
val_dataset = val_dataset.map(evaluate)

In [10]:
total = len(val_dataset)
matched = len(val_dataset.filter(lambda x: x["match"] == 1))
print("EM score:", (matched / total)*100, "%")


EM score: 25.447830101569714 %


In [ ]:
val_data.save_to_disk("results-nq")